In [2]:
import torch
import torchvision
import tqdm

#### Configuration

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda', index=0)

In [5]:
root = "data/"
logs = "lightning_logs/"

In [6]:
batch_size = 64
lr = 1e-3
num_workers = 4
epochs = 20

In [7]:
def accuracy(y, y_hat):
    return 100*((y==y_hat).sum().item())/y.size(0)

#### Data Pipeline

In [8]:
# transform
data_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [13]:
# download the data
train = torchvision.datasets.cifar.CIFAR10(root=root, train=True, transform=data_transform, download=True)
val = torchvision.datasets.cifar.CIFAR10(root=root, train=False, transform=data_transform, download=True)

0it [00:00, ?it/s]

100%|█████████▉| 169861120/170498071 [00:54<00:00, 6062847.66it/s]

Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


170500096it [01:10, 6062847.66it/s]                               

In [16]:
# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True, num_workers=num_workers)

### Pure PyTorch code

#### Model

In [78]:
class CNN(torch.nn.Module):
    
    def __init__(self, in_channels=3, n_classes=10):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, padding=1) # 32x32x3 -> 16x16x16
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) # 16x16x16 -> 8x8x32
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) # 4x4x64
        self.linear = torch.nn.Linear(in_features=4*4*64, out_features=124)
        self.output = torch.nn.Linear(in_features=124, out_features=n_classes)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = torch.nn.Dropout(p=0.25)
    
    def forward(self, x):
        x = self.maxpool(torch.nn.functional.relu(self.conv1(x)))
        x = self.maxpool(torch.nn.functional.relu(self.conv2(x)))
        x = self.maxpool(torch.nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4) # 4x4x64 -> 4*4*64
        x = torch.nn.functional.relu(self.linear(x))
        out = self.output(x)
        return out
    

In [79]:
model = CNN().to(device)

#### Optimizer, Loss and Tensorboard writer

In [80]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
criterian = torch.nn.CrossEntropyLoss()
writer = torch.utils.tensorboard.SummaryWriter()

#### The Good, the Bad and the Ugly training loop

In [81]:
%time
steps = 0

epoch_progress = tqdm.tqdm(total=epochs, desc="Epoch", position=0)

for epoch in range(epochs):
    epoch_train_loss = []
    epoch_train_acc = []
    epoch_val_loss = []
    epoch_val_acc = []
    
    batch_progress = tqdm.tqdm(total=len(train_loader), desc="Batch", position=0)
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        
        # calculate the loss and batch accuracy
        loss = criterian(outputs, labels)
        acc = accuracy(labels, torch.argmax(outputs, 1))
        
        
        # backpropagate the loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        if steps%200==0:
            print(f'epoch {epoch} | steps  {steps} | loss {loss.item()} | accuracy {acc}')
        
        epoch_train_loss.append(loss.item())
        epoch_train_acc.append(acc)
        
        writer.add_scalar("step_wise_loss", loss.item(), steps)
        writer.add_scalar("step_wise_acc", acc, steps)
        
        batch_progress.update(1)
        steps += 1
        
        
    
    val_progress = tqdm.tqdm(total=len(val_loader), desc="Validation", position=0)
    for i, (images, labels) in enumerate(val_loader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            # forward pass with no grads
            outputs = model(images)
            
            # loss and acc calcualtion
            loss = criterian(outputs, labels)
            acc = accuracy(labels, torch.argmax(outputs, 1))
            
            epoch_val_loss.append(loss.item())
            epoch_val_acc.append(acc)
            
            val_progress.update(1)
    
    # calculate the epochs wise train, val loss and acc
    t_loss = sum(epoch_train_loss)/len(epoch_train_loss)
    t_acc = sum(epoch_train_acc)/len(epoch_train_acc)
    v_loss = sum(epoch_val_loss)/len(epoch_val_loss)
    v_acc = sum(epoch_val_acc)/len(epoch_val_acc)
    
    # add to tensorboard
    writer.add_scalar("train_loss", t_loss, epoch)
    writer.add_scalar("train_acc", t_acc, epoch)
    writer.add_scalar("val_loss", v_loss, epoch)
    writer.add_scalar("val_acc", v_acc, epoch)
    
    print(f"Epoch {epoch} | train_loss {t_loss} | train_acc {t_acc} | val_loss {v_loss} | val_acc {v_acc}")
    
    epoch_progress.update(1)
        

Batch:   0%|          | 0/391 [00:00<?, ?it/s]

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


Batch:   4%|▍         | 15/391 [00:00<01:58,  3.18it/s]

epoch 0 | steps  0 | loss 2.3017399311065674 | accuracy 10.9375


Batch:  59%|█████▉    | 231/391 [00:01<00:01, 88.46it/s]

...

Validation: 100%|██████████| 391/391 [00:05<00:00, 70.27it/s] 

epoch 19 | steps  7800 | loss 0.5120217800140381 | accuracy 80.46875



Epoch: 100%|██████████| 20/20 [01:51<00:00,  5.42s/it].62it/s]

Epoch 19 | train_loss 0.48032914387905384 | train_acc 83.12380115089515 | val_loss 0.43226598389923115 | val_acc 85.15305306905371


Validation: 100%|██████████| 391/391 [00:13<00:00, 208.62it/s]

In [85]:
torch.save(model, "model.pth")

/home/macab/miniconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [86]:
net = torch.load("model.pth")

In [87]:
net

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear): Linear(in_features=1024, out_features=124, bias=True)
  (output): Linear(in_features=124, out_features=10, bias=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
)

### PyTorch-Lightning Code

In [9]:
import pytorch_lightning as pl

In [10]:
PATH = "lightning_logs/version_0/checkpoints/model"

In [34]:
class CNN(pl.LightningModule):
    
    def __init__(self, in_channels=3, n_classes=10):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, padding=1) # 32x32x3 -> 16x16x16
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) # 16x16x16 -> 8x8x32
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) # 8x8x32 -> 4x4x64
        self.linear = torch.nn.Linear(in_features=4*4*64, out_features=128)
        self.output = torch.nn.Linear(in_features=128, out_features=n_classes)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = torch.nn.Dropout(p=0.25)
        
    def forward(self, x):
        x = self.dropout(torch.nn.functional.relu(self.maxpool(self.conv1(x))))
        x = self.dropout(torch.nn.functional.relu(self.maxpool(self.conv2(x))))
        x = self.dropout(torch.nn.functional.relu(self.maxpool(self.conv3(x))))
        x = x.view(-1, 4*4*64)
        x = self.dropout(self.linear(x))
        out = self.output(x)
        return out
    
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=lr)
    
    def train_dataloader(self):
        dataset = torchvision.datasets.cifar.CIFAR10(root=root, train=True, transform=data_transform, download=True)
        loader = torch.utils.data.DataLoader(dataset=dataset,  batch_size=batch_size, shuffle=True, num_workers=num_workers)
        return loader
    
    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        
        # loss and accuracy
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        acc = accuracy(labels, torch.argmax(outputs, 1))
        log = {"loss":loss, "acc":torch.tensor(acc)}
        return {"loss":loss, "acc":torch.tensor(acc), "log":log}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["acc"] for x in outputs]).mean()
        log = {"epoch_loss":avg_loss, "epoch_acc":avg_acc}
        return {"epoch_loss":avg_loss, "epoch_acc":avg_acc, "log":log}
    
    def val_dataloader(self):
        dataset = torchvision.datasets.cifar.CIFAR10(root=root, train=False, transform=data_transform, download=True)
        loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
        return loader
    
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        acc = accuracy(labels, torch.argmax(outputs, 1))
        
        log = {"val_loss":loss, "val_acc":torch.tensor(acc)}
        return {"val_loss":loss, "val_acc":torch.tensor(acc), "log":log}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        
        log = {"val_loss":avg_loss, "val_acc":avg_acc}
        return {"val_loss":avg_loss, "val_acc":avg_acc, "log":log}
    
    def test_dataloader(self):
        dataset = torchvision.datasets.cifar.CIFAR10(root=root, train=False, transform=data_transform, download=True)
        loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
        return loader
    
    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        acc = accuracy(labels, torch.argmax(outputs, 1))
        
        log = {"test_loss":loss, "test_acc":torch.tensor(acc)}
        return {"test_loss":loss, "test_acc":torch.tensor(acc), "log":log}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        
        log = {"test_loss":avg_loss, "test_acc":avg_acc}
        return {"test_loss":avg_loss, "test_acc":avg_acc, "log":log}
    
        

##### Callbacks

In [35]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

In [36]:
early_stopping = EarlyStopping(monitor="val_loss", min_delta=0.01, patience=3)
model_checkpoint = ModelCheckpoint(filepath="lightning_logs/", monitor="val_loss")

/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory lightning_logs/ exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


##### Create model and trainer

In [37]:
# create pl trainer object 
trainer = pl.Trainer(
    gpus=[0],
    weights_summary=None,
    show_progress_bar=True,
    max_epochs=10,
#     callbacks=[early_stopping, model_checkpoint]
    
)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


In [38]:
model = CNN()

In [39]:
trainer.fit(model)

Files already downloaded and verified
Files already downloaded and verified      
Files already downloaded and verified
Epoch 2:   0%|          | 0/939 [00:00<?, ?it/s, loss=1.426, v_num=0]           

/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


Epoch 2:  83%|████████▎ | 782/939 [00:04<00:00, 169.64it/s, loss=1.282, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 3:  83%|████████▎ | 782/939 [00:04<00:00, 166.02it/s, loss=1.255, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 4:  83%|████████▎ | 782/939 [00:05<00:01, 155.72it/s, loss=1.178, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 5:  83%|████████▎ | 782/939 [00:04<00:00, 160.66it/s, loss=1.090, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 6:  83%|████████▎ | 782/939 [00:05<00:01, 150.40it/s, loss=1.066, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 7:  83%|████████▎ | 782/939 [00:05<00:01, 150.61it/s, loss=1.015, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 8:  83%|████████▎ | 782/939 [00:05<00:01, 155.12it/s, loss=1.089, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 9:  83%|████████▎ | 782/939 [00:04<00:00, 159.06it/s, loss=0.998, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 10:  83%|████████▎ | 782/939 [00:04<00:00, 160.92it/s, loss=0.969, v_num=0]
Validating: 0it [00:00

1

In [40]:
trainer.test()

Files already downloaded and verified
Testing:  94%|█████████▎| 147/157 [00:00<00:00, 200.66it/s]--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(69.2277), 'test_loss': tensor(0.8820, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 157/157 [00:01<00:00, 140.24it/s]


In [41]:
!tensorboard --logdir=lightning_logs

/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty